# Titanic

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier 
from sklearn.datasets import make_classification
from sklearn.preprocessing import OneHotEncoder
from sklearn.naive_bayes import GaussianNB

In [2]:
test = pd.read_csv("Data/test.csv")
train = pd.read_csv("Data/train.csv")

## Priprava podatkov

Mankajoče podatke v train odstranim, v test pa nadomestim z povprečno vrednostjo.

In [4]:
train = train[np.invert(np.isnan(train["Age"]))]
test["Age"] = test["Age"].fillna(np.mean(test["Age"]))
test["Fare"] = test["Fare"].fillna(np.mean(test["Fare"]))

Nov atribut minor:

In [5]:
def minors(x):
    minor = []
    for a in x["Age"]:
        if a < 16:
            minor = np.append(minor, 1)
        else:
            minor = np.append(minor, 0)
    x["Minor"] = minor

In [6]:
minors(train)
minors(test)

/home/matej/.local/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


Nova atributa na podlagi priimkov:
* Vsaj en prezivel z istim priimkom
* Vsi umrli z istim priimkom

In [7]:
def allDied(x, y, ista):
    allD = []
    lived = []
    allLived = []
    oneDead = []
    for index, row in x.iterrows():
        surname = row["Name"].split(",")[0]
        dead = 0
        alive = 0
        for index2, row2 in y.iterrows():
            if ista:
                if index != index2:
                    surname2 = row2["Name"].split(",")[0]
                    if surname2 == surname:
                        if row2["Survived"] == 1:
                            alive += 1
                        else:
                            dead += 1
            else:
                surname2 = row2["Name"].split(",")[0]
                if surname2 == surname:
                    if row2["Survived"] == 1:
                        alive += 1
                    else:
                        dead += 1
        if dead + alive < 2:
            allD = np.append(allD, 0)
            lived = np.append(lived, 0)
            allLived = np.append(allLived, 0)
            oneDead = np.append(oneDead, 0)
        else:
            if alive == 0:
                allD = np.append(allD, 1)
            else:
                allD = np.append(allD, 0)
            if alive >= 1:
                lived = np.append(lived, 1)
            else:
                lived = np.append(lived, 0)
            if dead == 0:
                allLived = np.append(allLived, 1)
            else:
                allLived = np.append(allLived, 0)
            if dead >= 1:
                oneDead = np.append(oneDead, 1)
            else:
                oneDead = np.append(oneDead, 0)
    x["SurnameAllDied"] = allD
    x["SurnameOneLived"] = lived
    x["SurnameOneDead"] = oneDead
    x["SurnameAllLived"] = allLived

In [8]:
allDied(train, train, True)
allDied(test, train, False)

## Pretvorba cabins

In [10]:
palube = []
for a in train["Cabin"]:
    if not pd.isna(a):
        palube = np.append(palube, a[0])
    else:
        palube = np.append(palube, a)
train["Deck"] = palube

In [11]:
palube = []
for a in test["Cabin"]:
    if not pd.isna(a):
        palube = np.append(palube, a[0])
    else:
        palube = np.append(palube, a)
test["Deck"] = palube

Tabele za napovedovanje:

In [12]:
def one_hot(name, x, y, bol):
    category = np.array([])
    for a in y[name]:
        if not pd.isna(a):
            if a not in category:
                category = np.append(category, a)
    category = np.sort(category)
    for c in category:
        if not pd.isna(c):
            current = []
            for a in y[name]:
                if a == c:
                    current = np.append(current, 1)
                else:
                    current = np.append(current, 0)
            if bol:
                f = name + ":" + str(c)
                x[f] = current
            else:
                x[c] = current

In [13]:
y = train["Survived"]
x = train[["Pclass", "Age", "Fare", "Minor", "SurnameAllDied", "SurnameOneLived", "SurnameAllLived", "SurnameOneDead", "SibSp", "Parch"]]
x["FamilySize"] = train["Parch"] + train["SibSp"]
one_hot("Sex", x, train, False)
one_hot("Deck", x, train, False)
one_hot("Embarked", x, train, True)
x = x.loc[:, x.columns != 'nan']
#enc = OneHotEncoder()
#enc.fit(train)

/tmp/ipykernel_7472/4009306914.py:5: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if a not in category:


In [14]:
xtest = test[["Pclass", "Age", "Fare", "Minor", "SurnameAllDied", "SurnameOneLived", "SibSp", "Parch"]]
xtest["FamilySize"] = test["Parch"] + test["SibSp"]
one_hot("Sex", xtest, test, False)
one_hot("Deck", xtest, test, False)
xtest = xtest.loc[:, xtest.columns != 'nan']
one_hot("Embarked", xtest, test, True)

/tmp/ipykernel_7472/4009306914.py:5: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if a not in category:


In [15]:
x

,Pclass,Age,Fare,Minor,SurnameAllDied,SurnameOneLived,SurnameAllLived,SurnameOneDead,SibSp,Parch,...,B,C,D,E,F,G,T,Embarked:C,Embarked:Q,Embarked:S
0,3,22.0,7.2500,0.0,0.0,0.0,0.0,0.0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1,38.0,71.2833,0.0,0.0,0.0,0.0,0.0,1,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,3,26.0,7.9250,0.0,0.0,0.0,0.0,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,1,35.0,53.1000,0.0,0.0,0.0,0.0,0.0,1,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,3,35.0,8.0500,0.0,0.0,0.0,0.0,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
885,3,39.0,29.1250,0.0,1.0,0.0,0.0,1.0,0,5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
886,2,27.0,13.0000,0.0,0.0,0.0,0.0,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
887,1,19.0,30.0000,0.0,0.0,1.0,0.0,1.0,0,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
889,1,26.0,30.0000,0.0,0.0,0.0,0.0,0.0,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


## Ocenjevanje atributov

In [16]:
from sklearn.feature_selection import mutual_info_classif

In [17]:
trenutni = mutual_info_classif(x,y)
for a in range(99):
    trenutni += mutual_info_classif(x,y)
trenutni/100

array([0.06415326, 0.02333312, 0.13279084, 0.01210138, 0.0274129 ,
       0.01151176, 0.00662885, 0.00595704, 0.0179364 , 0.01973202,
       0.03922997, 0.15199288, 0.15008996, 0.00777584, 0.0173213 ,
       0.00970746, 0.01202565, 0.01474219, 0.00642008, 0.00684792,
       0.008214  , 0.0182533 , 0.00573641, 0.01726741])

In [18]:
for a, b in sorted(zip(trenutni, x.columns)):
    print(b + ": " + str(a / 100))

Embarked:Q: 0.005736412816619818
SurnameOneDead: 0.005957035714917125
F: 0.00642007864114752
SurnameAllLived: 0.006628852784808768
G: 0.006847921757942675
A: 0.007775838598616341
T: 0.008213996146594998
C: 0.009707458168334536
SurnameOneLived: 0.01151175909591868
D: 0.01202564503087507
Minor: 0.012101379347520418
E: 0.014742193749123661
Embarked:S: 0.01726740861656779
B: 0.01732130077468309
SibSp: 0.01793639810751057
Embarked:C: 0.018253299319264636
Parch: 0.01973202163454215
Age: 0.023333118479695898
SurnameAllDied: 0.02741290195633948
FamilySize: 0.03922996900562584
Pclass: 0.06415325866622247
Fare: 0.1327908358348238
male: 0.15008995850952156
female: 0.15199287819060517


## Napovedovanje

### Random forest

In [35]:
#x3 = x[["Minor", "SurnameAllDied", "female"]]
#x3 = x[['Minor', 'SurnameAllDied', 'FamilySize', 'female', 'male', 'E']]
#x3 = x[['Minor', 'SurnameAllDied', 'FamilySize', 'male', 'E']] 0,77272
#x3 = x[['Age', 'Minor', 'SurnameAllDied', 'male', 'D', 'E', 'Embarked:S']] 0,71770
#x3 = x[['Minor', 'SurnameAllDied', 'female', 'Embarked:C', 'D']] 0,76555
#x3 = x[['Minor', 'SurnameAllDied', 'FamilySize', 'female', 'D', 'E']] 0,77033
x3 = x[['Pclass', 'Fare', 'Minor', 'SurnameAllDied', 'FamilySize', 'female', 'Embarked:C', 'D', 'E']]
x3test = xtest[['Pclass', 'Fare', 'Minor', 'SurnameAllDied', 'FamilySize', 'female', 'Embarked:C', 'D', 'E']]
clf = RandomForestClassifier()
clf.fit(x3,y)
r = clf.predict(x3test)

In [36]:
resoult = test[["PassengerId"]]
resoult["Survived"] = r
resoult.to_csv("Predictions/rfc42.csv",  index=False)

/home/matej/.local/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


### Naive Bayes

In [62]:
#x3 = x[["Minor", "SurnameAllDied", "female"]]
#x3 = x[['Minor', 'SurnameAllDied', 'FamilySize', 'female', 'male', 'E']]
#x3 = x[['Minor', 'SurnameAllDied', 'FamilySize', 'male', 'E']] 0,7751
#x3 = x[['Age', 'Minor', 'SurnameAllDied', 'FamilySize', 'female', 'E']] 0,7751
x3 = x[['Pclass', 'Age', 'Minor', 'SurnameAllDied', 'female', 'male', 'E']]
x3test = xtest[['Pclass', 'Age', 'Minor', 'SurnameAllDied', 'female', 'male', 'E']]
gnb = GaussianNB()
gnb.fit(x3,y)
tt = gnb.predict(x3test)

In [63]:
resoult2 = test[["PassengerId"]]
resoult2["Survived"] = tt
#resoult2 == resoult
resoult2.to_csv("Predictions/nbc4.csv",  index=False)

/home/matej/.local/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


## Napovedovanje + testiranje

### Random forest

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from itertools import combinations

In [21]:
maxS = 0.65
maxT = np.array([])
for s in range(4,11):
    for a in combinations(["Pclass", "Age", "Fare", "Minor", "SurnameAllDied", "FamilySize", "female", "Embarked:C", "B", "D", "E"], s):
        x1 = x[np.array(a)]
        ca = 0
        for i in range(3):
            x_train, x_test, y_train, y_test = train_test_split(x1, y, test_size=0.33)
            clf = RandomForestClassifier()
            clf.fit(x_train, y_train)
            r = clf.predict(x_test)
            ca += accuracy_score(y_test, r)
        current = ca / 3
        if maxS < current:
            maxS = current
            maxT = a
        #print(current)
    print(s)
    print(maxT)
    print(maxS)
    maxS = 0.65
    maxT = np.array([])

4
('Minor', 'SurnameAllDied', 'female', 'E')
0.8403954802259888
5
('Minor', 'SurnameAllDied', 'female', 'B', 'E')
0.8403954802259888
6
('Minor', 'SurnameAllDied', 'FamilySize', 'female', 'D', 'E')
0.8488700564971752
7
('Minor', 'SurnameAllDied', 'FamilySize', 'female', 'Embarked:C', 'D', 'E')
0.8446327683615819
8
('Age', 'Fare', 'Minor', 'SurnameAllDied', 'FamilySize', 'female', 'D', 'E')
0.824858757062147
9
('Pclass', 'Fare', 'Minor', 'SurnameAllDied', 'FamilySize', 'female', 'Embarked:C', 'D', 'E')
0.8234463276836158
10
('Pclass', 'Age', 'Fare', 'Minor', 'SurnameAllDied', 'FamilySize', 'female', 'B', 'D', 'E')
0.8149717514124294


In [140]:
maxS

0.8401937046004841

### Naive bayes

In [85]:
maxS = 0.65
maxT = np.array([])
for s in range(4,11):
    for a in combinations(["Pclass", "Age", "Fare", "Minor", "SurnameAllDied", "SurnameOneLived", "FamilySize", "female", "male", "B", "D", "E", "Embarked:C", "Embarked:S"], s):
        x1 = x[np.array(a)]
        ca = 0
        for i in range(10):
            x_train, x_test, y_train, y_test = train_test_split(x1, y, test_size=0.33)
            gnb = GaussianNB()
            gnb.fit(x_train, y_train)
            r = gnb.predict(x_test)
            ca += accuracy_score(y_test, r)
        current = ca / 10
        if maxS < current:
            maxS = current
            maxT = a
        #print(current)
    print(s)
    print(maxT)
    print(maxS)
    maxS = 0.65
    maxT = np.array([])

4
('Minor', 'SurnameAllDied', 'female', 'E')
0.8317796610169491
5
('Age', 'Minor', 'SurnameAllDied', 'male', 'E')
0.8381355932203389
6
('Age', 'Minor', 'SurnameAllDied', 'FamilySize', 'female', 'male')
0.8419491525423728
7
('Age', 'Minor', 'SurnameAllDied', 'male', 'D', 'E', 'Embarked:S')
0.83135593220339
8
('Pclass', 'Age', 'Minor', 'SurnameAllDied', 'FamilySize', 'female', 'male', 'Embarked:S')
0.8317796610169491
9
('Age', 'Minor', 'SurnameAllDied', 'FamilySize', 'female', 'male', 'B', 'E', 'Embarked:C')
0.8249999999999998
10
('Pclass', 'Fare', 'Minor', 'SurnameAllDied', 'SurnameOneLived', 'FamilySize', 'female', 'male', 'D', 'E')
0.8262711864406779


# Rezultat

Najboljšo oceno je dosegla metoda random forest, z naslednimi atributi: 'Minor', 'SurnameAllDied', 'FamilySize', 'male', 'E'.
Dosegel sem klasifikacijsko točnost: 0.77272. Kar se uvrsti med top 31%.